## Taxi Data Demonstration

In this demonstration we are using a portion of the [NYC Taxi dataset](http://www.andresmh.com/nyctaxitrips/) to show how to put together many of the features of the [declarative widgets](https://github.com/jupyter-incubator/declarativewidgets). You will see examples of:
* Importing 3rd party elements
* Using functions defined in Python
* Constructing input areas for the user
* Displaying results in tables, maps, charts
* Accessing DataFrames
* Modifying data from Python and affecting UI elements


<div class="alert alert-info" role="alert" style="margin-top: 10px">
<p><strong>Note</strong><p>

<p>To follow along on the demostration, it is best to execute cells as you read the Notebook (Shift-Enter). Do not use Run All.</p>
</div>

<div class="alert alert-info" role="alert" style="margin-top: 10px">
<p><strong>Note:</strong> This notebook was written using Python 3+.</p>
</div>

### Importing Elements <a name="import"></a>

One of the first things you need to do is import some elements. In this example, we are going to be using serveral of the elements provided by the [Polymer Catalog](https://elements.polymer-project.org/). With the `urth-core-import` extension to the `<link>` tag, you can import 3rd party components by setting the `package` attribute to the [Bower](http://bower.io/) package name. For example, the `<paper-slider>` element is defined in package `PolymerElements/paper-slider`.

In [ ]:
%%html
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-button/paper-button.html" is='urth-core-import' package='PolymerElements/paper-button'>
<link rel="import" href="urth_components/paper-card/paper-card.html" is='urth-core-import' package='PolymerElements/paper-card'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/google-map/google-map.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/google-map/google-map-marker.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>
<link rel="import" href="urth_components/urth-viz-chart/urth-viz-chart.html" is='urth-core-import'>

<style is="custom-style">
    google-map {
        height: 400px;
    }

    div.output_wrapper {
        z-index: 3;
    }

    paper-card.driverStats {
        margin-bottom: 10px;
        display: block;
        width: 50%;
    }

    paper-card.driverStats paper-slider {
        width: 100%;
    }

    paper-card.driverStats div.stat-label,
    paper-card.driverStats div.stat-value {
        width: 50px;
    }

    urth-viz-chart .nv-x .tick text {
        display: none;
    }

    paper-slider.blue {
      --paper-slider-knob-color: var(--paper-light-blue-500);
      --paper-slider-active-color: var(--paper-light-blue-500);
    }
    paper-slider.red {
      --paper-slider-knob-color: var(--paper-red-500);
      --paper-slider-active-color: var(--paper-red-500);
    }
    paper-slider.orange {
      --paper-slider-knob-color: var(--paper-orange-500);
      --paper-slider-active-color: var(--paper-orange-500);
      --paper-slider-pin-color: var(--paper-orange-500);
    }
    
    urth-viz-chart::shadow .nv-x .tick text {
        display: none;
    }
</style>

<div class="alert alert-info" role="alert" style="margin-top: 10px">
<p><strong>Note</strong><p>

<p>It will take some time the first time you run the cell above. That is because the Notebook server is indeed installing those element so that they are available to use here.</p>
</div>

In [ ]:
import pandas as pd

In [ ]:
tripsUrl = {
    'trips_1': 'https://ibm.box.com/shared/static/c58ytwlmffnofl3arcwc6axev6nocijp.csv',
    'trips_2': 'https://ibm.box.com/shared/static/lwsnp7gqwwjvl68mz751lbdoqe5hvhbo.csv',
    'trips_3': 'https://ibm.box.com/shared/static/iueg7r5k65l9jkz1ye40tkx7hcxxpxfq.csv',
    'trips_4': 'https://ibm.box.com/shared/static/1pd4vfp5mn4ge28dfsj32idbfzkvxg02.csv',
    'trips_5': 'https://ibm.box.com/shared/static/n1a88mgcm94h9mq8dcu5ss99j8xo9z1t.csv'
}

In [ ]:
def loadData(name:str):
    return pd.read_csv(tripsUrl[name])

### Using functions <a name="function"></a>

The cell above defines a simple function called `loadData()`. This function will load a CSV file containing taxi trip data. We can connect this function to UI elements with the help of the `<urth-core-function>` element. The element refers to the function by name using the `ref` attribute. It also takes arguments to the function as attributes and those can be set by other UI elements in the template. The cell below shows a simple UI consisting of a `selection` area and a `button`. 

The `<template>` element is using the `urth-core-bind` extension. This element behaves just like Polymer `dom-bind` extension with some additional goodness. More on that later... For more details, see the [documentation](https://www.polymer-project.org/1.0/docs/devguide/templates.html#dom-bind) on using Polymer's `dom-bind` `<template>`.

In [ ]:
%%html
<a name="loaddata"></a>
<template id="loadCard" is="urth-core-bind" channel="trips">
    <urth-core-function id="loadDataFunc" 
        ref="loadData" 
        arg-name="{{datafile}}" 
        result="{{thedata}}" 
        limit="50"
        is-ready="{{isloadready}}"></urth-core-function>
    <paper-card heading="Load Data" elevation="1">
        <div class="card-content">
            <paper-dropdown-menu label="Select Trips" selected-item-label="{{datafile}}" noink>
                <paper-menu class="dropdown-content" selected="[[datafile]]" attr-for-selected="label">
                    <template is="dom-repeat" items="[[tripOptions]]">
                        <paper-item label="[[item]]">[[item]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>
        </div>
        <div class="card-actions">
            <paper-button tabindex="0" disabled="{{!isloadready}}" onClick="loadDataFunc.invoke()">Load</paper-button>
        </div>
    </paper-card>
</template>

### Setting data

If you have not already noticed, the `Load Data` UI we create [above](#loaddata) has no options to select. That is because we have not given it a list of options. One way of setting that information is using a regular code cell.

In the cell below we are using the channel api to set data on the UI. This api is one of the added capabilities of using the `<template>` element with the `urth-core-bind` extension. Notice that the `<template>` element in the cell above has a `channel` attribute set to `trips`. This provides a namespace of sort for all the other binding varaibles defined using the `{{varname}}` syntax. From Python we can then set the value of the `{{tripOptions}}` variable.

Execute the cell below and see how there are options to choose from on the `<paper-dropdown-menu>` widget [above](#loaddata).

In [ ]:
from urth.widgets.widget_channels import channel
channel("trips").set("tripOptions", ["trips_1", "trips_2", "trips_3", "trips_4", "trips_5"])

In [ ]:
def center(lat1:float, long1: float, lat2: float, long2: float):
    return [(lat1+lat2)/2, (long1+long2)/2]

### Tables and Maps

The following cell will show the taxi trip data on a table and a map, but only if you've loaded data. You load the data with the `Load Data` UI created [above](#loaddata). Select a trip and press `'LOAD`. To show the data as a table, we are using the `<urth-viz-table>` element.

So where is the map? The map shows when selecting a row on the table. This capability is made possible by the `selection` attribute of the `<urth-viz-table>` element. 

We are also using the `center()` function defined above as a way to center the map between the pickup and dropoff location of the taxi trip.

In [ ]:
%%html
<template is="urth-core-bind" channel="trips">
    <urth-core-function
        ref="center" 
        arg-lat1="[[sel.11]]" 
        arg-long1="[[sel.10]]" 
        arg-lat2="[[sel.13]]" 
        arg-long2="[[sel.12]]" 
        result="{{thecenter}}" auto></urth-core-function>
    <template is="dom-if" if="{{thedata}}">
        <paper-card style="width: 100%;" heading="Trips Data" elevation="1">
            <div class="card-content">
                <urth-viz-table datarows='{{thedata.data}}' columns='{{thedata.columns}}' selection='{{sel}}' rows-visible="10"></urth-viz-table>
            </div>
            <div class="card-content">
                <template is="dom-if" if="{{sel}}">
                    <google-map id="blah" latitude="[[thecenter.0]]" longitude="[[thecenter.1]]" zoom="12">
                        <google-map-marker latitude="{{sel.11}}" longitude="{{sel.10}}" title="Pickup"></google-map-marker>
                        <google-map-marker latitude="{{sel.13}}" longitude="{{sel.12}}" title="Dropoff"></google-map-marker>
                    </google-map>
                </template>
            </div>
        </paper-card>
    </template>
</template>

In [ ]:
allfares = fares = pd.read_csv('https://ibm.box.com/shared/static/w5kzt0f6xgy0i0kadv41lxyqhhfudnh2.csv')

### Using DataFrame <a name="dataframe"></a>

The NYC Taxi data also contains a dataset for the fares collected for each trip. In the cell above we are loading this data into a Pandas DataFrame. Below, we are using the `<urth-core-dataframe>` element to interface with the Pandas DataFrame. Just like the `<urth-core-function>` element, the `ref` attribute is use to set the name of the DataFrame variable `fares`. 

Once connected, we can display the data using UI elements. In the cell below, we are using the `<urth-viz-table>` element to show the data.

In [ ]:
%%html
<template is="urth-core-bind" channel="faresChan">
    <urth-core-dataframe ref="fares" value="{{thedata}}" limit="100" auto></urth-core-dataframe>
    <paper-card style="width: 100%;" heading="Fares Data" elevation="1">
        <div class="card-content">
            <urth-viz-table datarows='{{thedata.data}}' columns='{{thedata.columns}}' rows-visible='10'></urth-viz-table>
        </div>
    </paper-card>
</template>

A nice feature of the `<urth-core-dataframe>` element is that it can detect cell execution and refresh its data in case the DataFrame it refers to is modified. Execute the cell below and notice how the contents of the table above changes.

In [ ]:
fares = allfares[fares[' payment_type']=='CRD']

In [ ]:
def filterTopEarners(filter = "All"):
    if (filter == "Credit"):
        _fares = allfares[allfares[' payment_type']=='CRD']
    elif (filter == "Cash"):
        _fares = allfares[allfares[' payment_type']=='CSH']
    else:
        _fares = allfares

    _topEarners = pd.DataFrame(_fares.groupby('medallion').sum().sort(' total_amount', inplace=False, ascending=False)[' total_amount']) 
    _topEarners['medallion'] = _topEarners.index
    
    return _topEarners[['medallion', ' total_amount']]

### Using charts <a name="chart"></a>

Now to something a bit more complex. Below is a cell using an `<urth-core-function>` element to connect to function `filterTopEarners()` defined above. The function returns a Pandas DataFrame with the top earners based on the specified payment type used for the taxi trip. This data is then displayed using an `<urth-viz-chart>` element.

Notice how the `<paper-dropdown-menu>` element defaults to `All`. This is because the `<urth-core-function>` elements knows when function arguments have default values and makes those values available to the `<template>`. Also, the `<paper-dropdown-menu>` has 2 other options. Like the above example, these are set from the python code right below.

Also notice the `<paper-slider>` element. It allows you to control how much of the returned DataFrame is made available to the `<urth-viz-chart>`.

In [ ]:
channel("faresChan").set("paymentOptions", ["All", "Cash", "Credit"])

In [ ]:
%%html
<a name="earners"></a>
<template id="topEarners" is="urth-core-bind" channel="faresChan">
    <urth-core-function ref="filterTopEarners" arg-filter="{{payType}}" result="{{top}}" limit="{{limit}}" auto></urth-core-function>
    <paper-card style="width: 100%;" heading="Top Earners" elevation="1">
        <div class="card-content">
            <paper-dropdown-menu label="Select Payment Type" noink>
                <paper-menu class="dropdown-content" selected="{{payType}}" attr-for-selected="label">
                    <template is="dom-repeat" items="[[paymentOptions]]">
                        <paper-item label="[[item]]">[[item]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>

            <urth-viz-chart type='bar' datarows='{{top.data}}' columns='{{top.columns}}' selection="{{sel}}"></urth-viz-chart>
            <div class="center horizontal layout">
                <div>Limit</div>
                <paper-slider min="5" max="50" step="5" pin snap value="{{limit}}"></paper-slider>
            </div>
        </div>
    </paper-card>
</template>

In [ ]:
def driverStats(medallions = []):
    colnames = ['medallion', 'trips', 'total_amount_avg', 'total_amount_min', 'total_amount_max']
    data = []

    for m in list(map((lambda med: med['x']), medallions)):
        stats = allfares[allfares['medallion']==m].groupby('medallion').describe()[' total_amount'].iloc[[0, 1, 3, 7]].values
        row = [m] + list(stats)
        data.append(row)

    return pd.DataFrame(data, columns=colnames)

def minMaxStats():
    data = []
    group = allfares[['medallion', ' total_amount']].groupby('medallion')

    the_mean = group.mean()
    the_max = group.max()
    the_min = group.min()

    return [the_mean.min()[0], the_mean.max()[0], the_max.min()[0], the_max.max()[0], the_min.min()[0], the_min.max()[0]]

### Build anything you want!

Finally, the cell below shows how much control the declarative elements give you to build different ways to show your data. We are creating a list of cards for each selection made on the bar chart [above](#earners) to show some individual statistics about the driver. Click on serveral bars and then scroll back down to see the resulting UI.

And how can the selection from up ther affect the content of the cell below? That is another feature of the `urth-core-bind` extension to the `<template>` element. As long as `<template>` elements are on the same `channel`, they can share `{{data}}`. 

In [ ]:
%%html
<template id="fareStatsTemplate" is="urth-core-bind" channel="faresChan">
    <urth-core-function ref="driverStats" arg-medallions="{{sel}}" result="{{stats}}" auto></urth-core-function>
    <urth-core-function id="minMaxStats" ref="minMaxStats" result="{{minMax}}" on-is-ready-changed ="init"></urth-core-function>
    <script>
        fareStatsTemplate.init = function(){
            minMaxStats.invoke()
        }
    </script>
    <template is="dom-repeat" items='{{stats.data}}' as='stat'>
        <paper-card class='driverStats' elevation="1">
            <div class="card-content">
                <div class="vertical-section">
                    <h4>Driver: <span>{{stat.0}}</span></h4><br/>
                    <div class="center horizontal layout">
                        <div class="stat-label">Avg</div>
                        <paper-slider pin value="{{stat.2}}" min='{{minMax.0}}' max='{{minMax.1}}'></paper-slider>
                        <div class="stat-value">$<span>{{stat.2}}</span></div>
                    </div>
                    
                    <div class="center horizontal layout">
                        <div class="stat-label">Min</div>
                        <paper-slider pin value="{{stat.3}}" min='{{minMax.4}}' max='{{minMax.5}}' class="orange"></paper-slider>
                        <div class="stat-value">$<span>{{stat.3}}</span></div>
                    </div>
                    
                    <div class="center horizontal layout">
                        <div class="stat-label">Max</div>
                        <paper-slider pin value="{{stat.4}}" min='{{minMax.2}}' max='{{minMax.3}}' class="red"></paper-slider>
                        <div class="stat-value">$<span>{{stat.4}}</span></div>
                    </div>
                </div>
            </div>
        </paper-card>
    </template>
</template>

For more detail examples of each of the elements, visit each of [these](../../examples) notebooks.